In [1]:
pip install deepface

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached deepface-0.0.79-py3-none-any.whl (49 kB)
  Using cached Flask-2.3.2-py3-none-any.whl (96 kB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.8 MB)
  Using cached fire-0.5.0-py2.py3-none-any.whl
  Using cached gdown-4.7.1-py3-none-any.whl (15 kB)
  Using cached retina_face-0.0.13-py3-none-any.whl (16 kB)
  Using cached mtcnn-0.1.1-py3-none-any.whl (2.3 MB)
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached Werkzeug-2.3.4-py3-none-any.whl (242 kB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.2.3
    Uninstalling Werkzeug-2.2.3:
      Successfully uninstalled Werkzeug-2.2.3
  Attempting uninstall: blinker

In [2]:
pip install opencv-contrib-python

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached opencv_contrib_python-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (67.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install image2base64

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached image2base64-0.0.5-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [7]:
# imports
from deepface import DeepFace
import os
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
import uuid
import pandas as pd
from image2base64.converters import base64_to_rgb, rgb2base64
import json
import traceback
import re
import time
import requests

In [33]:
# models
path = r"models/EDSR_x2.pb"
sr = cv2.dnn_superres.DnnSuperResImpl_create()
sr.readModel(path)
sr.setModel("edsr", 2)

# directories
dir = r"pictures"
json_path = r"json/persons.json"
db_path = r"stored_faces_db"

# Image resizing options
crop_increase_percent = 25
standard_height = 155
standard_width = 120

uuid_regex = r'([\da-f]{8}-[\da-f]{4}-[\da-f]{4}-[\da-f]{4}-[\da-f]{12})'

# db api
URL_TABLE = "https://gc7da7be5da2e70-g833jueqvvi5nhsa.adb.eu-frankfurt-1.oraclecloudapps.com/ords/admin/persons/"
URL_MODULE = "https://gc7da7be5da2e70-g833jueqvvi5nhsa.adb.eu-frankfurt-1.oraclecloudapps.com/ords/admin/operations/"

In [16]:
def add_face_to_json(face_dict, cropped_cv2):
    new_id = uuid.uuid4()
    cv2.imwrite(r"stored_faces_db/" + str(new_id) + ".jpg", cropped_cv2)
    
    upscale_img = upscale(cropped_cv2)
    
    data = {
        "id": str(new_id),
        "cropped_img": rgb2base64(upscale_img),
        "age": face_dict["age"],
        "gender": face_dict["dominant_gender"],
        "emotion": face_dict["dominant_emotion"],
        "first_name": "",
        "last_name": "",
        "is_present": "True",
    }
    
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    response = requests.post(URL_TABLE, data=json.dumps(data), headers=headers)
    
    print("Added new face to oracle db")

In [1]:
def refresh_face_in_json(face_dict, id, cropped_cv2):
    upscale_img = upscale(cropped_cv2)
    
    payload = {
        "in_id": id,
        "in_cropped_img": rgb2base64(upscale_img),
        "in_age": face_dict["age"], 
        "in_gender": face_dict["dominant_gender"], 
        "in_emotion": face_dict["dominant_emotion"]
    }
    
    headers = {'Content-type': 'application/json'}
    response = requests.post(URL_MODULE + "refresh_person", json.dumps(payload), headers=headers)
    print(response.status_code)
    
    print("Refresh face in oracle db")

In [10]:
def upscale(cropped_cv2):
    
    normalized_size_cropped = cv2.resize(cropped_cv2, (standard_width, standard_height))
    sr_upscale_np =  sr.upsample(normalized_size_cropped)     
                                         
    return Image.fromarray(sr_upscale_np)

In [34]:
def reset_is_present():
    response = requests.post(URL_MODULE + "reset_is_present")
    print("Reset is_present")
    print(response.status_code)

200


In [17]:
start = time.time()
# load persons.json and initialize dict
if os.path.getsize(json_path) ==0:
    p_dict = {"persons": []}
else: 
    with open("json/persons.json", "r") as file:
        p_dict = json.load(file)

for person in p_dict["persons"]:
    person["is_present"] = False
    
# loop       
for file in os.listdir(dir):
    try:
        img = cv2.imread(dir + "/" + file)
        
        if img is None:
            continue
        
        analyze = DeepFace.analyze(img_path=img, actions = ['age', 'gender', 'race', 'emotion'], 
                                   detector_backend= 'mtcnn', enforce_detection=False, silent=True)

        for face in analyze:

            x = face['region'].get('x')
            y = face['region'].get('y')
            w = face['region'].get('w')
            h = face['region'].get('h')

            w_increase = int(w * crop_increase_percent / 100)
            h_increase = int(h * crop_increase_percent / 100)

            x_new = x - w_increase // 2
            y_new = y - h_increase // 2
            w_new = w + w_increase
            h_new = h + h_increase
            
            cropped = cv2.cvtColor(img[y_new:y_new + h_new, x_new:x_new + w_new, :], cv2.COLOR_BGR2RGB)
            
            # Deepface find
            if len(os.listdir(db_path)) <= 1:    
                print("DB empty")
                add_face_to_json(face, cropped)
            else:
                list_of_df = DeepFace.find(img_path=cropped, db_path=db_path, detector_backend="retinaface",
                                           enforce_detection=False)   
                df = list_of_df[0]
                if df.shape[0] > 0:
                    path_to_dbface = list_of_df[0]['identity'].iloc[0]
                    face_id = re.findall(uuid_regex, path_to_dbface)[0]
                    refresh_face_in_json(face_dict=face, id=face_id, cropped_cv2=cropped)
                else:
                    print("No such face in DB")
                    add_face_to_json(face_dict=face, cropped_cv2=cropped)
                                    
    except Exception as e:
        print(file)
        traceback.print_exc()
        continue
        
with open(r"json/persons.json", "w") as file:
    json.dump(p_dict, file)
print(time.time()-start)

2/2 [==============================] - 0s 5ms/step
There are  1  representations found in  representations_vgg_face.pkl
find function lasts  0.5954031944274902  seconds
pexels-andrea-piacquadio-3777920.jpg
1/1 [==============================] - 0s 18ms/step


Traceback (most recent call last):
  File "/tmp/ipykernel_302/1169572276.py", line 51, in <module>
    refresh_face_in_json(face_dict=face, id=face_id, cropped_cv2=cropped)
  File "/tmp/ipykernel_302/3171347954.py", line 5, in refresh_face_in_json
    edit["cropped_face"] = rgb2base64(upscale_img)
TypeError: 'NoneType' object does not support item assignment


2/2 [==============================] - 0s 4ms/step
There are  1  representations found in  representations_vgg_face.pkl
find function lasts  0.5901691913604736  seconds
No such face in DB
<Response [201]>
Added new face to oracle db
There are  1  representations found in  representations_vgg_face.pkl
find function lasts  0.5405588150024414  seconds
No such face in DB
<Response [201]>
Added new face to oracle db
There are  1  representations found in  representations_vgg_face.pkl
find function lasts  0.5723743438720703  seconds
No such face in DB
<Response [201]>
Added new face to oracle db
There are  1  representations found in  representations_vgg_face.pkl
find function lasts  0.5046391487121582  seconds
No such face in DB
<Response [201]>
Added new face to oracle db
There are  1  representations found in  representations_vgg_face.pkl
find function lasts  0.5991671085357666  seconds
No such face in DB
<Response [201]>
Added new face to oracle db
There are  1  representations found in 

In [ ]:
pip install confluent

In [ ]:
from confluent_kafka.admin import AdminClient, NewTopic

a = AdminClient({'bootstrap.servers': '192.168.70.40:9092'})
#new_topics = [NewTopic(topic, num_partitions=1, replication_factor=1) for topic in ["toAnalyze", "toUI"]]

#a.create_topics(new_topics)

print(a.list_topics().topics)

